# Ambiente

### Instalar librerías y reiniciar kernel.

In [1]:
pip install mysql-connector

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install pyarrow

In [4]:
#pip install pyspark

In [5]:
#pip install fastparquet

### Importaciones

In [6]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import *
#import json
#import pyarrow.parquet as pq
#import pyarrow
#from pyspark.sql import SparkSession
#from fastparquet import ParquetFile
#import fastparquet

### Crear acceso a engine de mysql e inspeccionar.

In [7]:
engine = create_engine('mysql+mysqlconnector://test:test123@db/test', echo = True)

In [8]:
pd.read_sql("select current_date", con=engine)

2021-02-04 07:24:31,667 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-02-04 07:24:31,669 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 07:24:31,674 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-02-04 07:24:31,677 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 07:24:31,684 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-02-04 07:24:31,687 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 07:24:31,694 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-02-04 07:24:31,696 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 07:24:31,701 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-02-04 07:24:31,703 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 07:24:31,710 INFO sqlalchemy.engine.base.Engine DESCRIBE `select current_date`
2021-02-04 07:24:31,711 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 07:24:31,716 INFO sqlalc

,current_date
0,2021-02-04


In [9]:
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        for column in inspector.get_columns(table_name, schema=schema):
            print("Column: %s" % column)

2021-02-04 07:24:31,815 INFO sqlalchemy.engine.base.Engine SHOW schemas
2021-02-04 07:24:31,817 INFO sqlalchemy.engine.base.Engine {}
schema: information_schema
2021-02-04 07:24:31,823 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `information_schema`
2021-02-04 07:24:31,825 INFO sqlalchemy.engine.base.Engine {}
schema: test
2021-02-04 07:24:31,836 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `test`
2021-02-04 07:24:31,837 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 07:24:31,889 INFO sqlalchemy.engine.base.OptionEngine SHOW CREATE TABLE `test`.`time_series_covid19_confirmed_global`
2021-02-04 07:24:31,892 INFO sqlalchemy.engine.base.OptionEngine {}
Column: {'name': 'index', 'type': BIGINT(display_width=20), 'default': None, 'comment': None, 'nullable': True, 'autoincrement': False}
Column: {'name': 'Province/State', 'type': TEXT(), 'default': None, 'comment': None, 'nullable': True}
Column: {'name': 'Country/Region', 'type': TEXT(), 'default': None, 'comment

In [10]:
print(engine.table_names())

2021-02-04 07:24:32,008 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `test`
2021-02-04 07:24:32,010 INFO sqlalchemy.engine.base.Engine {}
['time_series_covid19_confirmed_global', 'unconfirmed_transactions', 'vaccinations_per_country']


# ETL 4

## Extraer

In [11]:
#table = pq.read_table('data.parquet')
#table.to_pandas()

In [ ]:
#df = pd.read_parquet('sources/data.parquet', engine='fastparquet')
#pd.read_parquet('example_pa.parquet', engine='pyarrow')
df = pd.read_parquet(r'sources/data.parquet', engine='auto', columns=None, use_nullable_dtypes=False)

In [ ]:
df.dtypes

## Transformar

In [ ]:
u = df.select_dtypes(object)
df[u.columns] = u.apply(
    lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

## Load al engine

In [ ]:
create_table_query = pd.io.sql.get_schema(df.reset_index(), 'test.data_parquet', con=engine)
print(create_table_query)
type(create_table_query)

In [ ]:
with open("data_parquet.sql", "w") as text_file:
    text_file.write(create_table_query)

In [ ]:
if not engine.dialect.has_table(engine, "test.data_parquet"):  # If table don't exist, Create.
    print("test.data_parquet no existe")
    df.to_sql('data_parquet', con=engine, schema="test", if_exists='append')

In [ ]:
with engine.begin() as transaction:
    transaction.execute('DELETE FROM test.data_parquet')
    df.to_sql('data_parquet', con=transaction, schema='test', if_exists='append', index=False)